# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [60]:
pkl_path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_6s_all.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [61]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [62]:
train_data

{'melspectrogram': array([[1.1545551e-05, 1.2176995e-05, 1.3009772e-05, ..., 4.4375400e-07,
         2.2230227e-08, 1.8851988e-08],
        [8.5422990e-08, 9.0774186e-08, 1.0014884e-07, ..., 1.3420139e-06,
         4.4213351e-07, 4.1396237e-07],
        [6.1857563e-06, 6.2330437e-06, 6.3144412e-06, ..., 1.1637738e-06,
         1.5971284e-07, 1.3454519e-07],
        ...,
        [7.1480241e-07, 7.4554231e-07, 7.9745979e-07, ..., 1.4340408e-06,
         1.3201277e-06, 1.2859776e-06],
        [6.0994887e-07, 6.2010758e-07, 6.3716243e-07, ..., 4.0761867e-07,
         1.7535218e-07, 1.6836647e-07],
        [6.8865182e-07, 6.9677486e-07, 7.1000841e-07, ..., 3.5803578e-06,
         1.9428686e-08, 1.4416809e-08]], dtype=float32),
 'mfcc': array([[-1.00000000e+00, -2.42030799e-01, -4.80754524e-01, ...,
          3.10237170e-03,  1.18157947e-02, -6.41129969e-04],
        [-1.00000000e+00, -2.10041642e-01, -3.97155881e-01, ...,
         -2.89330794e-03,  1.39303915e-02, -7.43062468e-03],
        

In [63]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [64]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [65]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [66]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [67]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [68]:
display(train_y.shape)
display(train_y[:15])

(7105,)

array([ 1, 12, 12, 18,  0,  6,  9,  2, 15,  9, 15,  0,  9,  2, 12])

In [69]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(7105, 60)

array([1.24520002e-05, 1.25051074e-05, 1.25950455e-05, 1.27242620e-05,
       1.28970751e-05, 1.31336319e-05, 1.37108636e-05, 1.88041904e-05,
       5.99552332e-05, 1.67028586e-04, 4.74756642e-04, 1.58886088e-03,
       6.27184939e-03, 2.35568155e-02, 4.17131148e-02, 6.10713549e-02,
       1.19916029e-01, 1.96485519e-01, 2.68517882e-01, 2.14806095e-01,
       1.67547256e-01, 1.78057253e-01, 2.27615058e-01, 1.75201088e-01,
       2.41183847e-01, 3.06942105e-01, 3.03498685e-01, 2.94426799e-01,
       3.40946317e-01, 3.37458670e-01, 3.53853822e-01, 2.59901941e-01,
       2.10388824e-01, 2.14529440e-01, 1.95060119e-01, 1.76715493e-01,
       2.30201513e-01, 4.12996024e-01, 4.52444166e-01, 3.97506714e-01,
       2.89133191e-01, 2.81264603e-01, 2.05479860e-01, 2.03507066e-01,
       1.73288599e-01, 1.01112396e-01, 8.12784433e-02, 9.11540985e-02,
       7.94092789e-02, 4.16857973e-02, 3.03183608e-02, 2.60504503e-02,
       1.85266640e-02, 1.22771878e-02, 5.58216823e-03, 1.38317782e-03,
      

'mfcc'

(7105, 20)

...

(7105, 12)

array([0.52648836, 0.43961033, 0.36696267, 0.3728558 , 0.36008734,
       0.3787417 , 0.55697906, 0.660865  , 0.62423116, 0.55163324,
       0.5511625 , 0.55658233], dtype=float32)

'rms'

(7105, 1)

array([0.01086454], dtype=float32)

In [70]:
display(val_y.shape)
display(val_y[:15])

(1896,)

array([15,  2,  2,  2,  0, 18,  2,  7, 17,  3,  2, 19,  2, 19,  0])

In [71]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(1896, 60)

array([8.24601466e-06, 8.32974547e-06, 8.47288811e-06, 8.68162442e-06,
       8.96583515e-06, 9.36863034e-06, 1.00850721e-05, 1.25935021e-05,
       2.51408219e-05, 6.64327672e-05, 2.89875490e-04, 1.02110556e-03,
       2.76797637e-03, 8.43508821e-03, 2.13416293e-02, 4.79326099e-02,
       8.08759779e-02, 1.35495916e-01, 1.74270004e-01, 2.24694893e-01,
       2.28031531e-01, 2.24797517e-01, 2.18499169e-01, 2.30090767e-01,
       3.15125734e-01, 4.35118824e-01, 4.80999261e-01, 2.99557686e-01,
       1.89516738e-01, 1.80220544e-01, 1.52036324e-01, 1.19603343e-01,
       1.68151662e-01, 4.02095467e-01, 3.09555948e-01, 2.56599545e-01,
       1.89351156e-01, 1.19057849e-01, 1.20452479e-01, 1.85614154e-01,
       1.82802498e-01, 9.73533317e-02, 1.01837352e-01, 9.06173885e-02,
       6.37171119e-02, 4.57593799e-02, 5.37593290e-02, 2.55445074e-02,
       2.75482386e-02, 2.31455751e-02, 9.23997071e-03, 7.62971677e-03,
       6.52957009e-03, 5.14294999e-03, 1.67173590e-03, 2.47013988e-04,
      

'mfcc'

(1896, 20)

...

(1896, 12)

array([0.31514877, 0.33721167, 0.49286336, 0.53746825, 0.42874166,
       0.43082845, 0.43903908, 0.5120638 , 0.50912267, 0.41824663,
       0.31891856, 0.32977086], dtype=float32)

'rms'

(1896, 1)

array([0.00875111], dtype=float32)

## **Random Forest Model**

In [147]:
train_results = {}
val_results = {}

### **With all the features**

In [148]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['rms'], train_features['melspectrogram']), axis=1)

training_features.shape

(7105, 93)

In [149]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['rms'], val_features['melspectrogram']), axis=1)

validation_features.shape

(1896, 93)

Fit the model with training data

In [150]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [151]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.41139240506329117


In [152]:
train_yhat_result = rf.predict(training_features)

print('Training classification Report \n')
print(classification_report(train_y, train_yhat_result))

Training classification Report 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1017
           1       1.00      1.00      1.00       727
           2       1.00      1.00      1.00       501
           3       1.00      1.00      1.00       462
           4       1.00      1.00      1.00        66
           5       1.00      1.00      1.00       116
           6       1.00      1.00      1.00       123
           7       1.00      1.00      1.00       403
           8       1.00      1.00      1.00       108
           9       1.00      1.00      1.00       197
          10       1.00      1.00      1.00       282
          11       1.00      1.00      1.00       107
          12       1.00      1.00      1.00       718
          13       1.00      1.00      1.00       183
          14       1.00      1.00      1.00       265
          15       1.00      1.00      1.00       403
          16       1.00      1.00      1.00     

In [153]:
val_yhat_result = rf.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.39      0.73      0.51       272
           1       0.22      0.55      0.31       118
           2       0.80      0.21      0.33       476
           3       0.56      0.95      0.70        92
           4       0.00      0.00      0.00        49
           5       0.27      0.30      0.29        20
           6       0.00      0.00      0.00        18
           7       0.66      0.74      0.70       111
           8       0.17      0.14      0.15        37
           9       0.69      0.77      0.73        65
          10       0.47      0.32      0.38        75
          11       0.00      0.00      0.00        37
          12       0.23      0.39      0.29        89
          13       0.17      0.03      0.05        31
          14       0.06      0.16      0.09        19
          15       0.49      0.28      0.36       136
          16       0.77      0.21      0.33   

### **With MFCCs only**

In [154]:
training_features = train_features['mfcc']

training_features.shape

(7105, 20)

In [155]:
validation_features = val_features['mfcc']

validation_features.shape

(1896, 20)

In [156]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [157]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3391350210970464


### **With Mel-Spectrogram Only**

In [158]:
training_features = train_features['melspectrogram']

training_features.shape

(7105, 60)

In [159]:
validation_features = val_features['melspectrogram']

validation_features.shape

(1896, 60)

In [160]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [161]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.31276371308016876


## **With Chroma Only**

In [162]:
training_features = train_features['chroma']

training_features.shape

(7105, 12)

In [163]:
validation_features = val_features['chroma']

validation_features.shape

(1896, 12)

In [164]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [165]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.22732067510548523


## **With MFCCs and Mel-Spectrogram**

In [166]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(7105, 80)

In [167]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(1896, 80)

In [168]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [169]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3850210970464135


## **With MFCCs and Chroma**

In [170]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(7105, 32)

In [171]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(1896, 32)

In [172]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [173]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3570675105485232


## **With MFCCs and RMS**

In [174]:
training_features = np.concatenate((train_features['mfcc'], train_features['rms']), axis=1)

training_features.shape

(7105, 21)

In [175]:
validation_features = np.concatenate((val_features['mfcc'], val_features['rms']), axis=1)

validation_features.shape

(1896, 21)

In [176]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [177]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.350210970464135


## **With Chroma and RMS**

In [178]:
training_features = np.concatenate((train_features['chroma'], train_features['rms']), axis=1)

training_features.shape

(7105, 13)

In [179]:
validation_features = np.concatenate((val_features['chroma'], val_features['rms']), axis=1)

validation_features.shape

(1896, 13)

In [180]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [181]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.24208860759493672


## **With Mel-Spectrogram and Chroma**

In [182]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(7105, 72)

In [183]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(1896, 72)

In [184]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [185]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.33069620253164556


## **With Mel-Spectrogram and RMS**

In [186]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['rms']), axis=1)

training_features.shape

(7105, 61)

In [187]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['rms']), axis=1)

validation_features.shape

(1896, 61)

In [188]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [189]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_rms'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_rms'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3206751054852321


## **With Mel-Spectrogram, MFCCs, RMS**

In [190]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['rms'], train_features['mfcc']), axis=1)

training_features.shape

(7105, 81)

In [191]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['rms'], val_features['mfcc']), axis=1)

validation_features.shape

(1896, 81)

In [192]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [193]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_rms_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_rms_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.38343881856540085


## **With Mel-Spectrogram, MFCCs, Chroma**

In [194]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(7105, 92)

In [195]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(1896, 92)

In [196]:
rf = RandomForestClassifier(n_estimators=50, criterion='entropy')
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', n_estimators=50)

In [197]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3697257383966245


# Review results from all models

In [198]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(2)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(2)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
3,chroma,1.0,0.23
7,chroma_rms,1.0,0.24
2,melspectrogram,1.0,0.31
9,melspectrogram_rms,1.0,0.32
8,melspectrogram_chroma,1.0,0.33
1,mfcc,1.0,0.34
6,mfcc_rms,1.0,0.35
5,mfcc_chroma,1.0,0.36
11,melspectrogram_chroma_mfcc,1.0,0.37
10,melspectrogram_rms_mfcc,1.0,0.38
